# Time Series Modeling

## First - Data Set Up

Data is Bitcoin data at 1-min intervals from select exchanges, Jan 2012 to March 2021 - [source](https://www.kaggle.com/mczielinski/bitcoin-historical-data)

In [ ]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller

from pmdarima.utils import decomposed_plot
from pmdarima.arima import decompose

In [ ]:
# If you need pmdarima:
# http://alkaline-ml.com/pmdarima/
# !pip install pmdarima

In [ ]:
# Read df
df = pd.read_csv('data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

In [ ]:
# Check the shape of the data
df.shape

In [ ]:
# Check the first 10 rows...
df.head(10)

In [ ]:
# Woah! Nulls! Is that going to be a problem?
df.isna().sum()
# 1231878 / 3997697 = .3ish (about 1/3 of our data is null)

In [ ]:
# Before worrying about nulls, let's change the timestamp
# By setting unit to seconds, we get the date from the unix time
df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='s')

In [ ]:
# Let's go ahead and set the index to be a datetime index
df.set_index('Timestamp', inplace=True)

In [ ]:
# Now let's look at how that impacted the time/index
df.head()

In [ ]:
df.tail()

In [ ]:
# Let's visualize all of our columns, to get a sense of the data
for col in df.columns:
    plt.figure(figsize=(16,6))
    df[col].plot()
    plt.title(col)
    plt.show()

So we can see the general trends in our data, and we can see that, if those nulls are having an impact, it's not apparent in the plot!

Regarding the Open / High / Low / Close plots - do you see much of a difference?

Also! This is why we change our data to use the datetime object as the index - makes EVERYTHING easier!

## Changing the frequency of our data 

Also called downsampling or upsampling, depending on whether you're going to a less frequent or more frequent point in time.

[Here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling) is a reference for resampling based on time frequency. (you can find the actual codes you can use as arguments in the resample function [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)).

In [ ]:
# Downsampling to a daily cadence
df_daily = df.resample('D').mean()

In [ ]:
df_daily.head()

In [ ]:
# Let's check if we still have any nulls
df_daily.isna().sum()

In [ ]:
# Looking into where those nulls are...
df_daily.loc[df_daily['Open'].isna() == True]

In [ ]:
# Let's visualize just one column to see what changed on a daily scale
plt.figure(figsize=(16,6))
df_daily['Open'].plot()
plt.title('Average Open Price - Daily')
plt.show()

I don't see much of a change, just fewer data points - it's probably safe to say that we didn't lose much data by downsampling to a daily mean.

But what about downsampling to monthly?

In [ ]:
# Here we're downsampling to month end, denoted by 'M'
# If we wanted month start, we could use 'MS'
df_monthly = df.resample('M').mean()
df_monthly.head()

In [ ]:
# Again, visualizing the Average Opening price
plt.figure(figsize=(16,6))
df_monthly['Open'].plot()
plt.title('Average Open Price - Monthly')
plt.show()

And what about quarterly?

In [ ]:
# Here we're downsampling to month end, denoted by 'M'
# If we wanted month start, we could use 'MS'
df_quarterly = df.resample('Q').mean()
df_quarterly.head()

In [ ]:
# Again, visualizing the Average Opening price
plt.figure(figsize=(16,6))
df_quarterly['Open'].plot()
plt.title('Average Open Price - Quarterly')
plt.show()

So, as would make sense, as you change the frequency of your data it changes the granularity (level of detail) that's conveyed.

Before we move forward, let's grab a single year from this data, 2017, at the daily frequency to explore.

In [ ]:
df_2017 = df[df.index.year == 2017]

In [ ]:
df_2017_daily = df_2017.resample('D').mean()

In [ ]:
df_2017_daily.head()

In [ ]:
# Again, visualizing the Average Opening price
plt.figure(figsize=(16,6))
df_2017_daily['Open'].plot()
plt.title('Average Open Price - 2017')
plt.show()

## Stationarity

Introduction to stationarity from [_Forecasting: Principles and Practice_](https://otexts.com/fpp2/stationarity.html):

> "A stationary time series is one whose properties do not depend on the time at which the series is observed.14 Thus, time series with trends, or with seasonality, are not stationary — the trend and seasonality will affect the value of the time series at different times. On the other hand, a white noise series is stationary — it does not matter when you observe it, it should look much the same at any point in time.
>
> "Some cases can be confusing — a time series with cyclic behaviour (but with no trend or seasonality) is stationary. This is because the cycles are not of a fixed length, so before we observe the series we cannot be sure where the peaks and troughs of the cycles will be.
>
> "In general, a stationary time series will have no predictable patterns in the long-term. Time plots will show the series to be roughly horizontal (although some cyclic behaviour is possible), with constant variance."

And here's a [useful blog post](https://towardsdatascience.com/stationarity-in-time-series-analysis-90c94f27322) on the subject, where I found the below demonstrative image:

![Examples of stationary and non-stationary processes, from the above medium blog](https://miro.medium.com/max/1400/1*tkx0_wwQ2JT7pSlTeg4yzg.png)

In [ ]:
# Let's  get the rolling mean and rolling standard deviation, for the 
# opening price, using a 5-day window

roll_mean = df_2017_daily['Open'].rolling(window=5, center=False).mean()
roll_std = df_2017_daily['Open'].rolling(window=5, center=False).std()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
ax.plot(df_2017_daily['Open'], color='blue',label='Average Daily Opening Price')
ax.plot(roll_mean, color='red', label='Rolling 5-Day Mean')
ax.plot(roll_std, color='black', label='Rolling 5-Day Standard Deviation')
ax.legend()
fig.tight_layout()

Do you think this data is stationary? Why or why not?

 - 
 

There's a test for this!

> **Augumented Dickey-Fuller test**: a hypothesis test, where we reject the null hypothesis (that a time series is non-stationary) if the test-statistic is less than the critical value

[Documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.adfuller.html) for the Dickey-Fuller test in StatsModels

So let's write out our null and alternative hypotheses (remember these??):

Ho = 

Ha = 

In [ ]:
# Feed in our Open column, since the test function expects a series:
adfuller(df_2017_daily['Open'])

Let's interpret the output of this test:

- 


In [ ]:
# Now that we've determined whether the data is stationary, let's decompose it

# Using the decompose function from the arima model
# Need to feed it an array, hence the .values attribute
decomposed = decompose(df_2017_daily['Open'].values, 'additive', m=20)

In [ ]:
# Plotting the pieces using the arima model again
decomposed_plot(decomposed, figure_kwargs={'figsize': (16, 10)})
plt.show()

In [ ]:
# Now let's check the difference

df_2017_daily_diff = df_2017_daily['Open'].diff()

In [ ]:
# Now grabbing the rolling mean and std for the difference
diff_roll_mean = df_2017_daily_diff.rolling(window=5, center=False).mean()
diff_roll_std = df_2017_daily_diff.rolling(window=5, center=False).std()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
ax.plot(df_2017_daily_diff, color='blue',label='Difference')
ax.plot(diff_roll_mean, color='red', label='Rolling Mean')
ax.plot(diff_roll_std, color='black', label='Rolling Standard Deviation')
ax.legend()
fig.tight_layout()

More or less stationary?

- 


In [ ]:
# What if we logged the data?
logged_2017_daily = np.log1p(df_2017_daily['Open'])

In [ ]:
# Now grabbing the rolling mean and std for the difference
log_roll_mean = logged_2017_daily.rolling(window=5, center=False).mean()
log_roll_std = logged_2017_daily.rolling(window=5, center=False).std()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
ax.plot(logged_2017_daily, color='blue',label='Difference')
ax.plot(log_roll_mean, color='red', label='Rolling Mean')
ax.plot(log_roll_std, color='black', label='Rolling Standard Deviation')
ax.legend()
fig.tight_layout()

More or less stationary?

- 


Any further ideas? 

- 


## Now - Time to Model!

## Featuring: ARMA

### Autoregressive (AR)


An autoregressive (AR) model is when a value from a time series is regressed on previous values from the same time series.

In words, the mathematical idea is the following:

$$ \text{Today = Constant + Slope} \times \text{Yesterday + Noise} $$

Or, mathematically:
$$\large Y_t = \mu + \phi * Y_{t-1}+\epsilon_t$$

### Moving Average (MA)

The Moving Average (MA) model can be described as the weighted sum of today's and yesterday's noise.

In words, the mathematical idea is the following:

$$ \text{Today = Mean + Noise + Slope} \times \text{Yesterday's Noise} $$

Or, mathematically:
$$\large Y_t = \mu +\epsilon_t + \theta * \epsilon_{t-1}$$


Some notes on these, based on the formulas:
- If the slope is 0, the time series is a white noise model with mean $\mu$
- If the slope is not 0, the time series is autocorrelated and depends on the previous white noise process
- Bigger slope means bigger autocorrelation
- When there is a negative slope, the time series follow an oscillatory process

In [ ]:
# New dataset who dis
# Monthly Google search trends for 'taxes' in the US
df_taxes = pd.read_csv('data/google-trends_taxes_us.csv')

# Some quick clean up
df_taxes.columns = ['counts']
df_taxes = df_taxes.iloc[1:]
df_taxes['counts'] = df_taxes['counts'].str.replace('<1', '0').astype(int)
df_taxes.index = pd.to_datetime(df_taxes.index)

In [ ]:
df_taxes.head()

In [ ]:
df_taxes.tail()

## ARMA Modeling 

Once you determine if your time series is stationary, you can model. There are 4 key steps: 
1. Model Identification - where you determine the properties of a time series then chose a structural form. Remember you're treating the data as a series of random variables. The basic types of ARIMA models are: 
    - AutoRegressive(AR)
    - Moving Average(MA) 
    - Autoregressive Moving Average(ARMA)
    - AutoRegressive Integrated Moving Average 
A time series may be primarily an autoregressive, moving average or combination of both. To identify which it is, you need to plot 2 key functions. 
    > Sample Autocorrelation Function(ACF) 
    > Sample Partial Autocorrelation Function(Partial ACF)
    
2. Parameter Estimation - Once you have identified the form of an ARIMA model, the next step is to estimate the coefficients or parameters of the model. You can use Regression and MLE to do this. 

3. Model Checking - The most widely used information criterion(checking the quality of your model) for Time series is AIC. You can compare different models with different numbers of lagged terms, white noise terms and how many times the time series was differenced and choose the model with the lowest AIC. 

4. Forecasting - Once the model is estimated you can forecast future values with the predict function. 

![](https://www.statisticshowto.com/wp-content/uploads/2016/11/lag-plot-linear.png)

## Autocorrelation and Partial Autocorrelation
The ACF shows the correlations between the elements of a time series as a function of their lags. The partial ACF shows the correlations between the elements of a time series for each lag, holding constant the impact of all other lags.

>The basic idea of autocorrelation is simple: See how a series correlates with a "lagged" version of itself. If my sequence is $S_0 = (x_0, x_1, x_2, ... , x_n)$, then I can measure the Pearson correlation(multicollinearity) between the first $n-k + 1$ terms of $S_0$ and $S_{lag} = (x_k, x_{k+1}, x_{k+2}, ... , x_n)$.


### Autocorrelation Function Plots

https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

> - "The **autocorrelation function** is a function that represents autocorrelation of a time series as a function of the time lag."
- The autocorrelation function tells interesting stories about trends and seasonality. For example, if the original time series repeats itself every five days, you would expect to see a spike in the autocorrelation function at 5 days.



In [ ]:
import statsmodels.graphics.tsaplots as tsa

tsa.plot_acf(df_taxes['counts'],lags=52);

In [ ]:
# Another view
plt.figure(figsize=(20, 4))
pd.plotting.autocorrelation_plot(df_taxes['counts']);

The horizontal bands represent condfidence intervals, which are calculated by taking relevant z-scores of the standard normal distribution and dividing by the square root of the number of observations. What do these intervals mean? - anything outside confidence interval means not due to chance - reject null. 


### Partial-Autocorrelation Function Plot


> "The **partial autocorrelation function** can be interpreted as a regression of the series against its past lags.
 
 > It helps you come up with a possible order for the auto regressive term. The terms can be interpreted the same way as a standard linear regression, that is the contribution of a change in that particular lag while holding others constant. "

The idea behind partial Autocorrelation is to compare a series to a lagged version of itself while abstracting away from intermediate values. In effect, this amounts to exploring the correlations among residuals

In [ ]:
tsa.plot_pacf(df_taxes['counts'],lags=50);

# ARIMA MODELS:


## The ARIMA Time Series Model

One of the most common methods used in time series forecasting is known as the ARIMA model, which stands for **AutoregRessive Integrated Moving Average**. ARIMA is a model that can be fitted to time series data in order to better understand or predict future points in the series.

Let's have a quick introduction to ARIMA. The ARIMA forecasting for a stationary time series is nothing but a linear (like a linear regression) equation. The predictors depend on the parameters (p,d,q) of the ARIMA model:

### Number of AR (Auto-Regressive) terms (p): 

`p` is the auto-regressive part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to rain tomorrow if it has been raining for past 3 days. AR terms are just lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).

### Number of Differences (d):

`d` is the **Integrated** component of an ARIMA model. This value is concerned with the amount of differencing as it identifies the number of lag values to subtract from the current observation. Intuitively, this would be similar to stating that it is likely to rain tomorrow if the difference in amount of rain in the last *n* days is small. 

### Number of MA (Moving Average) terms (q): 

`q` is the moving average part of the model which is used to set the error of the model as a linear combination of the error values observed at previous time points in the past. MA terms form lagged forecast errors in prediction equation. For instance if q is 5, the predictors for x(t) will be e(t-1)….e(t-5) where `e(i)` is the difference between the moving average at ith instant and actual value.

These three distinct integer values, (p, d, q), are used to parametrize ARIMA models. Because of that, ARIMA models are denoted with the notation `ARIMA(p, d, q)`. Together these three parameters account for seasonality, trend, and noise in datasets:

* `(p, d, q)` are the non-seasonal parameters described above.
* `(P, D, Q)` follow the same definition but are applied to the seasonal component of the time series. 
* The term `s` is the periodicity of the time series (4 for quarterly periods, 12 for yearly periods, etc.).

A detailed article on these parameters is available [HERE](https://www.quantstart.com/articles/Autoregressive-Integrated-Moving-Average-ARIMA-p-d-q-Models-for-Time-Series-Analysis).

The seasonal ARIMA method can appear daunting because of the multiple tuning parameters involved. In the next section, we will describe how to automate the process of identifying the optimal set of parameters for the seasonal ARIMA time series model.

What you generally will try to do for any time series analysis is:

- Detrend your time series using differencing. ARMA models represent stationary processes, so we have to make sure there are no trends in our time series
- Look at ACF and PACF of the time series
- Decide on the AR, MA, and order of these models
- Fit the model to get the correct parameters and use for prediction

In [ ]:
# metrics
from sklearn import metrics

# Note - we're back to regression metrics!
def report_metrics(y_true, y_pred):
    print("Explained Variance:\n\t", metrics.explained_variance_score(y_true, y_pred))
    print("MAE:\n\t", metrics.mean_absolute_error(y_true, y_pred))
    print("RMSE:\n\t", metrics.mean_squared_error(y_true, y_pred, squared=False))
    print("r^2:\n\t", metrics.r2_score(y_true, y_pred))

In [ ]:
# Back to taxes
# Let's create a time period tracker
df_taxes.insert(0, 't', range(len(df_taxes)))

# Let's create a target area
df_taxes['future'] = (df_taxes.index.year > 2017).astype('int')

# Now plot
plt.figure(figsize=(10,6))
df_taxes.loc[df_taxes.future == 0, 'counts'].plot()
df_taxes.loc[df_taxes.future == 1, 'counts'].plot()
plt.show()

In [ ]:
X_train = df_taxes.loc[df_taxes.future == 0, 't'].values.reshape(-1, 1)
X_test = df_taxes.loc[df_taxes.future == 1, 't'].values.reshape(-1, 1)
y_train = df_taxes.loc[df_taxes.future == 0, 'counts'].values
y_test = df_taxes.loc[df_taxes.future == 1, 'counts'].values

In [ ]:
# naive approach
pred_val = y_train[len(y_train)-1]
y_pred = [pred_val] * len(y_test)
pred_val

In [ ]:
X_test

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_taxes.loc[df_taxes.future == 0]['counts'])
plt.plot(df_taxes.loc[df_taxes.future == 1]['counts'])

plt.plot(X_test, y_pred)

plt.show()

In [ ]:
report_metrics(y_test, y_pred)

Thoughts? 

- 


In [ ]:
# linear trend approach
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_trend = lr.predict(X_train)

In [ ]:
y_pred

In [ ]:
df_taxes.loc[df_taxes.future == 1].index

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(df_taxes['counts'])
plt.plot(df_taxes.loc[df_taxes.future == 0].index, y_trend)
plt.plot(df_taxes.loc[df_taxes.future == 1].index, y_pred, color='green')
plt.show()

In [ ]:
report_metrics(y_test, y_pred)

### PMDArima

In [ ]:
import pmdarima
pmdarima.auto_arima?
